In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_theme(style='darkgrid',palette='dark')
import matplotlib.pyplot as plt
import warnings as warn
warn.filterwarnings('ignore')
from surprise.model_selection import KFold
from surprise import SVD
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split,cross_validate
from surprise import Dataset
from surprise import Reader
import math
from collections import defaultdict

In [2]:
df1=pd.read_csv('phone_user_review_file_1.csv',encoding="latin-1")
df2=pd.read_csv('phone_user_review_file_2.csv',encoding="latin-1")
df3=pd.read_csv('phone_user_review_file_3.csv',encoding="latin-1")
df4=pd.read_csv('phone_user_review_file_4.csv',encoding="latin-1")
df5=pd.read_csv('phone_user_review_file_5.csv',encoding="latin-1")
df6=pd.read_csv('phone_user_review_file_6.csv',encoding="latin-1")

In [3]:
print(f"Dataframe 1 size = {df1.shape}")
print(f"Dataframe 1 size = {df2.shape}")
print(f"Dataframe 1 size = {df3.shape}")
print(f"Dataframe 1 size = {df4.shape}")
print(f"Dataframe 1 size = {df5.shape}")
print(f"Dataframe 1 size = {df6.shape}")
print(f"Final Dataframe rows = {df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]+df6.shape[0]}")

Dataframe 1 size = (374910, 11)
Dataframe 1 size = (114925, 11)
Dataframe 1 size = (312961, 11)
Dataframe 1 size = (98284, 11)
Dataframe 1 size = (350216, 11)
Dataframe 1 size = (163837, 11)
Final Dataframe rows = 1415133


In [4]:
arr1=df1.values
arr2=df2.values
arr3=df3.values
arr4=df4.values
arr5=df5.values
arr6=df6.values

In [5]:
arr=np.concatenate((arr1,arr2,arr3,arr4,arr5,arr6))

In [6]:
df=pd.DataFrame(arr,columns=df1.columns)

In [7]:
print(f"Dataframe size = {df.shape}")

Dataframe size = (1415133, 11)


In [8]:
df.describe().T

,count,unique,top,freq
phone_url,1415133,5556,/cellphones/samsung-galaxy-s-iii/,17093
date,1415133,7728,7/18/2016,3244
lang,1415133,22,en,554746
country,1415133,42,us,318435
source,1415133,331,Amazon,728471
domain,1415133,384,amazon.com,214776
score,1351644.0,86.0,10.0,656239.0
score_max,1351644.0,1.0,10.0,1351644.0
extract,1395772,1321353,#NAME?,667
author,1351931,801103,Amazon Customer,76978


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415133 entries, 0 to 1415132
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   phone_url  1415133 non-null  object
 1   date       1415133 non-null  object
 2   lang       1415133 non-null  object
 3   country    1415133 non-null  object
 4   source     1415133 non-null  object
 5   domain     1415133 non-null  object
 6   score      1351644 non-null  object
 7   score_max  1351644 non-null  object
 8   extract    1395772 non-null  object
 9   author     1351931 non-null  object
 10  product    1415132 non-null  object
dtypes: object(11)
memory usage: 118.8+ MB


In [10]:
print(f"Columns with nan values:\n{df.isna().sum()} \n---------------\nColumns with null values:\n{df.isnull().sum()}")

Columns with nan values:
phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64 
---------------
Columns with null values:
phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64


In [11]:
#finding columns with null values
def null_columns(dataframe):
    cols=dataframe.columns
    nulls=[]
    for i in cols:
        if dataframe[i].isna().sum()!=0:
            nulls.append(i)
    return nulls    
    
#finding the medians for imputation of missing values
def find_fills(dataframe):
    cols=null_columns(dataframe)
    filler=[]
    for i in cols:
        try:
            val=dataframe[i].value_counts()[:1].sort_values(ascending=False).index[0]
        except IndexError:
            val=dataframe[i].median()
        finally:
            filler.append(val)
    return(filler)
    
#filling the missing values with the most occuring values

def filler(dataframe):
    cols=null_columns(dataframe)
    fills=find_fills(dataframe)
    for i in range(len(cols)):
        val=fills[i]
        dataframe[cols[i]].fillna(val,inplace=True)
    print(dataframe.isna().sum())

In [12]:
filler(df)

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
extract      0
author       0
product      0
dtype: int64


In [13]:
df['score']=df['score'].astype('float64')
df['score']=df['score'].apply(lambda x:round(x))

In [14]:
df=df.drop_duplicates(keep='first')

In [15]:
print(f"Final Dataframe Shape:{df.shape}")

Final Dataframe Shape:(1408655, 11)


In [16]:
#Selecting 1M Records for analysis
df1=df.sample(n=1000000,random_state=612)
print(f"Dataframe Shape:{df1.shape}")

Dataframe Shape:(1000000, 11)


In [17]:
#Dropping irrelevant features
df1=df1.drop(['phone_url','date','lang','score_max','country','source','domain'],axis=1)
print(f"Dataframe Shape:{df1.shape}")

Dataframe Shape:(1000000, 4)


In [18]:
df2=df1.copy(deep=True)
df2.shape

(1000000, 4)

In [19]:
df1.head()

,score,extract,author,product
1219720,10,Nadat ik reeds voor een goed jaar en half een ...,Steven,Samsung Galaxy S II White
801292,10,Also ich bin wirklich zufrieden mit der Tusche...,Sinika90,Samsung Galaxy Express I8730
1158058,10,Definitely worth every penny and I recommend t...,J. Higgins,"Nokia N86 Unlocked Phone with 8 MP Camera, Aut..."
518835,8,"...was soll ich sagen, es tut, was es soll...i...",einem Kunden,"ZTE Blade A452 Smartphone, 12,7 cm (5 Zoll) Di..."
1307580,9,I've had my enV for 5 days and in a nutshell: ...,daisy33,enV VX-9900


## Popularity based Recommendation for popular mobile Phones

In [20]:
product_grp=df1.groupby(['product'])
ratings=pd.DataFrame(product_grp['score'].min())
ratings.rename({'score':'min_ratings'},axis=1,inplace=True)
ratings['max_ratings']=product_grp['score'].max()
ratings['num_ratings']=product_grp['score'].count()
ratings['mean_ratings']=product_grp['score'].mean()

In [21]:
df_ratings=ratings.reset_index()

In [22]:
#Data with ratings count more than 50
df_ratings=df_ratings[(df_ratings['min_ratings']!=df_ratings['max_ratings']) & (df_ratings['num_ratings']>50)]

print("Top 10 popular Products by number of purchases")
num_purchases=df_ratings.sort_values(ascending=False,by='num_ratings')

display(num_purchases['product'].head(10))
print("\n")
print("Top 10 popular Products with best ratings")

rates=df_ratings.sort_values(ascending=False,by='mean_ratings')

display(rates['product'].head(10))

Top 10 popular Products by number of purchases


21823       Lenovo Vibe K4 Note (White,16GB)
21822      Lenovo Vibe K4 Note (Black, 16GB)
31967            OnePlus 3 (Graphite, 64 GB)
31968           OnePlus 3 (Soft Gold, 64 GB)
37712           Samsung Galaxy Express I8730
15743            Huawei P8lite zwart / 16 GB
21826    Lenovo Vibe K5 (Gold, VoLTE update)
41086        Samsung Galaxy S6 zwart / 32 GB
28182                 Nokia 5800 XpressMusic
21828    Lenovo Vibe K5 (Grey, VoLTE update)
Name: product, dtype: object



Top 10 popular Products with best ratings


20844                            LG X power (Ð¸Ð½Ð´Ð¸Ð³Ð¾)
20842                              LG X power (Ð±ÐµÐ»ÑÐ¹)
20843                    LG X power (Ð·Ð¾Ð»Ð¾ÑÐ¸ÑÑÑÐ¹)
25716    Motorola Smartphone Motorola Novo Moto G Dual ...
4862                            Apple iPhone 7 4,7" 256 GB
55526    Ð¡Ð¾ÑÐ¾Ð²ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½ ASUS ZenFone 2 D...
38356                                    Samsung Galaxy J7
46386    Smartphone Motorola Moto G, 3G, Android 4.3, D...
4563                           Apple iPhone 6s 4,7" 128 GB
45894          Sim Free iPhone 7 32GB Mobile Phone - Black
Name: product, dtype: object

### Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final dataset.

In [23]:
product_users=df1.groupby(['product','author'])
top_50=pd.DataFrame(product_users['score'].count())

#renaming column for better understanding
top_50.rename({'score':'num_ratings'},axis=1,inplace=True)

top_50[top_50['num_ratings']>50].sort_values(by='num_ratings',ascending=False)
print(f"Number of products having more than 50 ratings and users who have given more than 50 ratings is {top_50[top_50['num_ratings']>50].shape[0]}")
display(top_50[top_50['num_ratings']>50].sort_values(by='num_ratings',ascending=False).head(10))

Number of products having more than 50 ratings and users who have given more than 50 ratings is 369


,,num_ratings
product,author,
"Lenovo Vibe K4 Note (White,16GB)",Amazon Customer,2171
"Lenovo Vibe K4 Note (Black, 16GB)",Amazon Customer,1754
"OnePlus 3 (Graphite, 64 GB)",Amazon Customer,1283
"OnePlus 3 (Soft Gold, 64 GB)",Amazon Customer,1202
"Lenovo Vibe K5 (Gold, VoLTE update)",Amazon Customer,1094
"Lenovo Vibe K5 (Grey, VoLTE update)",Amazon Customer,972
"Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)",Amazon Customer,807
"Lenovo Vibe K5 (Silver, 16GB)",Amazon Customer,649
YU Yuphoria YU5010A (Black+Silver),Amazon Customer,636


### Identify the most rated features.

In [24]:
ratings_mean_count = pd.DataFrame(df1.groupby('extract')['score'].count().sort_values(ascending=False)) 
ratings_mean_count['rating_counts'] = pd.DataFrame(df1.groupby('extract')['score'].count().sort_values(ascending=False))  
ratings_mean_count.head(5) 

,score,rating_counts
extract,,
#NAME?,14024,14024
Excellent product,185,185
Very good product,119,119
Excelente producto,87,87
very good product,77,77


### Identify the users with most number of reviews.

In [25]:
users = pd.DataFrame(df1.groupby('author')['score'].count().sort_values(ascending=False)) 
users.rename({'score':'number_of_ratings'},axis=1,inplace=True)
users.head(10)

,number_of_ratings
author,
Amazon Customer,98350
Cliente Amazon,13653
e-bit,5974
Client d'Amazon,5487
Amazon Kunde,3296
Anonymous,1915
einer Kundin,1890
einem Kunden,1350
unknown,1197


### Splitting Dataframe into testing,training set

In [26]:
#Reducing samples to 5k
df2=df2.sample(random_state=500,n=5000,ignore_index=True)

#Creating a user defined dataset

itemID=df2['product'].values.tolist()
userID=df2['author'].values.tolist()
rating=df2['score'].values.tolist()
ratings_dict = {'itemID': itemID,
                'userID': userID,
                'rating': rating}
rating = pd.DataFrame(ratings_dict)

display(rating.head())

#Reader to read the ratings on rating_scale param scale from min to max rating
reader = Reader(rating_scale=(min(rating['rating']), max(rating['rating'])))

#loading the dataset 
data = Dataset.load_from_df(rating[['userID', 'itemID', 'rating']], reader)

#Splitting the dataset
trainset, testset = train_test_split(data, test_size=.30)


,itemID,userID,rating
0,Samsung GALAXY Note II 16GB,Anatomy92,10
1,Samsung Galaxy Ace (S5830i) - Smartphone libre...,fsedeno,8
2,Smartphone Nokia Lumia 730 Dual Chip Desbloque...,vincetodoh,10
3,Samsung Guru GT-E1200 (Indigo Blue),Mith,10
4,Nokia C3-00 SIM Free Smartphone,Robert Walker,6


## collaborative filtering using item-item and user-user based Recommendation for popular mobile Phones

In [27]:
#Training the user based model based on KNN
algo1 = KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})
algo1.fit(trainset)

#Running on the test dataset
test_pred1 = algo1.test(testset)


#Training the item based model based on KNN
algo2 = KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': False})
algo2.fit(trainset)

#Running on the test dataset
test_pred2 = algo2.test(testset)



Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


## Collaborative filtering based on SVD

In [28]:
df3=df1.copy(deep=True)
df3=df3.sample(n=5000)
item=df3['product'].values.tolist()
user=df3['author'].values.tolist()
score=df3['score'].values.tolist()
score_dict={'score':score,
           'item':item,
           'user':user}
score_df=pd.DataFrame(score_dict)

#Reader to read the ratings on rating_scale param scale from min to max rating
reader = Reader(rating_scale=(min(score_df['score']), max(score_df['score'])))

#loading the dataset 
data = Dataset.load_from_df(score_df[['user', 'item', 'score']], reader)

In [29]:
#getting the training data from the dataset
train=data.build_full_trainset()

#Fitting the SVD on trainset with best parameters
algo3=SVD()

algo3.fit(train)

#Building testset from the trainset which does not have a rating for the user and item combination.
test=train.build_anti_testset()

#Make prediction
prediction3=algo3.test(test)

In [30]:
#Finding the average rating for the test users
def average_rating(dataset):
    sum=0
    count=0
    for i in dataset:
        sum+=i.r_ui
        count+=1
    return(sum/count)

#Finding RMSE Score for SVD based Model
def score(dataset):
    sum=0
    count=0
    avg_score=average_rating(dataset)
    for i in dataset:
        diff=0
        square=0
        estm=i.est
        diff=avg_score-estm
        square=math.pow(diff,2)
        sum+=square
        count+=1
    print(f"Average Rating for all of the predicted users :={avg_score}")
    return(math.sqrt(sum/count))
print(f"RMSE Score for Collaborative model using SVD:={score(prediction3)}")
print(f"RMSE Score for Collaborative model using item-item based filtering:={score(test_pred2)}")
print(f"RMSE Score for Collaborative model using user-user based filtering:={score(test_pred1)}")

Average Rating for all of the predicted users :=8.140000000548978
RMSE Score for Collaborative model using SVD:=0.332261215829479
Average Rating for all of the predicted users :=8.045333333333334
RMSE Score for Collaborative model using item-item based filtering:=0.4816088120369681
Average Rating for all of the predicted users :=8.045333333333334
RMSE Score for Collaborative model using user-user based filtering:=0.40496361081277915


#### The SVD based Colloborative filtering gives best accuracy score 
#### User-User based colloborative filtering gives much better results as compared to item-item filtering

#### Tuning the SVD model with K fold validation

In [40]:
# Defining Kfold
kf=KFold(n_splits=5,shuffle=True)

#Declaring SVD algorithm
algo4=SVD()

#Splitting data into train test
for train,test in kf.split(data):
    algo4.fit(train)
    prediction4=algo4.test(test)
print(f"RMSE Score for Collaborative model using k-fold SVD :={score(prediction4)}")      

Average Rating for all of the predicted users :=8.043
RMSE Score for Collaborative model using k-fold SVD :=0.40834020292701423


In [42]:
# Getting top 5 Best mobile recommendation for the users

def top_recommendations(prediction,N):
    
    top_items=defaultdict(list)
    for uid,iid,true_r,est,_ in prediction:
        top_items[uid].append((iid,est))
    
    for uid,ratings in top_items.items():
        ratings.sort(key= lambda x:x[1],reverse=True)
        top_items[uid]=ratings[:N]
    return top_items
recommendations=top_recommendations(prediction4,5)

In [43]:
# Getting top 5 Best mobile recommendation for a specific User 
def get_products(recommendations,user):
    for uid,items in recommendations.items():
        if uid=='Amazon Customer':
            item_list=items
    for product in item_list:
        display(product[0])
get_products(recommendations,"carol jones")        

'Nokia N8'

'OnePlus 3T (Soft Gold, 6GB RAM + 64GB memory)'

'OnePlus 3T (Gunmetal, 6GB RAM + 64GB memory)'

'LG KM900 Arena'

'Nokia 603'

#### In what business scenario you should use popularity based Recommendation Systems ?

Ans: Popularity based recommendation system is more useful when the user is new and there is no prior information about the buying habits of the users. So we can show them the most popular products in our store. This would prevent us from cold start issue and atleast a scope getting some idea after the user makes a purchase.

##### In what business scenario you should use CF based Recommendation Systems ?

Ans: Colloborative filtering recommendation would be used when theuser has atleast a profile and he has some prior history of some products and has rated some of the products and/or there are users who are similar to this target user and has similar taste of products thereby facilitating us to make a common recommendation to these both users.

#### What other possible methods can you think of which can further improve the recommendation for different users ?

Ans: The other possible way is to use either of the following approaches.

1. Use a combination of Popularity based algorithm and CF based method.
2. Getting the contents that are explcitly selected by the user and when a profile is made for the user make a colloborative approach to find similar users and recommend the products shared by both the users forming a unique combination.
3. Based on the frequent trend in buying pattern of the user try and recommend the Similar products based on the market basket approach thereby increasing the sale of the main product aswell as the other item in combination.

#### Improvements and Recommendations

* As evident from the dataset there is much anamoly/irregularity with respect to the Users,Description/Reviews about the product and the Similar variant of the Product forming a repetative combination was found that certainly led to the decrease in the model performance.
* There are a large number of users which are named as Anonymous or termed as Customer of the site the purchase made. It could be further analysed and asked for more refinemenet to make a better understanding.
* There should be a provision of the Country, Company attribute which would help in making a broader classification of the clusters for the products 